In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/dongmin/userdata/open-score-string-quartets


/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import sys
from pathlib import Path
from collections import Counter, defaultdict
from operator import itemgetter
from tempfile import NamedTemporaryFile

import math
import random

import json
import csv
import strictyaml as syaml

from tqdm import tqdm
import matplotlib.pyplot as plt

import cv2
import numpy as np
import partitura as pt

from bs4 import BeautifulSoup
import requests

## Load scores.yaml

In [3]:
data_dir = Path('data')

In [4]:
with open(data_dir / 'scores.yaml') as f:
  metadata = syaml.load(f.read())

metadata = metadata.data

In [5]:
len(metadata.keys())

122

## crawl PDF URLs

In [6]:
request_pdf_format = 'https://imslp.org/wiki/Special:ImagefromIndex/%(imslp_id)s/xe11'
disclaimer_url_format = 'https://imslp.org/wiki/Special:IMSLPDisclaimerAccept/%(imslp_id)s'

In [25]:
for mscore_id, obj in tqdm(metadata.items()):
  imslp_id = obj['imslp'][1:]
  
  res = requests.get(request_pdf_format % { 'imslp_id': imslp_id })
  soup = BeautifulSoup(res.text, 'lxml')
  
  if 'Disclaimer' in soup.title.string:
    res = requests.get(disclaimer_url_format % { 'imslp_id': imslp_id })
    soup = BeautifulSoup(res.text, 'lxml')
  
  if '.pdf' in res.url:
    pdf_url = res.url
  else:
    pdf_url = soup.select_one('#sm_dl_wait').get('data-id')
  
  metadata[mscore_id]['imslp_url'] = pdf_url

  0%|          | 0/122 [00:00<?, ?it/s]

 17%|█▋        | 21/122 [01:04<05:11,  3.08s/it]


AttributeError: 'NoneType' object has no attribute 'get'

In [27]:
soup

<html><body><p>You have reached this message because the site ripping ban script has been triggered. Site ripping is forbidden; repeated offenders will be banned indefinitely.<br/>
<br/>
I sincerely apologize if you have reached this message without using a site ripper; the site ripping ban script is very much a work in progress, and it is inevitable that there will be mistaken bans. You will be automatically unbanned after a short period of time, though this will increase exponentially should you get banned again. Also, it would be very much appreciated if you can drop an e-mail to feldmahler {at} imslp.org explaining the circumstances, along with your IP address (which is <b>163.239.15.35</b>), so that I can decrease the incidences of mistaken banning. Due to the volume of e-mails I receive, I may not respond to your report, but all reports are very much appreciated.<br/>
<br/>
<b>Please do not reload this page often, because every reload refreshes the ban length.</b> Instead, come b

In [30]:
syaml.as_document(metadata).as_yaml()

"7313978:\n  path: Andrée,_Elfrida/String_Quartet_in_A_major\n  name: String Quartet in A major\n  link: https://musescore.com/openscore-string-quartets/scores/7313978\n  imslp: '#415720'\n  set_id: 5108429\n  imslp_url: https://vmirror.imslp.org/files/imglnks/usimg/8/8a/IMSLP415720-PMLP674153-Andree_Elfrida_Kvartett_Strakar_Autograf.pdf\n13744399:\n  path: Arriaga,_Juan_Crisóstomo_de/String_Quartet_No.1_in_D_minor\n  name: String Quartet No.1 in D minor\n  link: https://musescore.com/openscore-string-quartets/scores/13744399\n  imslp: '#33857'\n  set_id: 5116346\n  imslp_url: https://s9.imslp.org/files/imglnks/usimg/c/cb/IMSLP33857-PMLP29989-Violin_I-1-.pdf\n7383977:\n  path: Arriaga,_Juan_Crisóstomo_de/String_Quartet_No.3_in_E-flat_Major\n  name: String Quartet No.3 in E-flat Major\n  link: https://musescore.com/openscore-string-quartets/scores/7383977\n  imslp: '#576844'\n  set_id: 5116350\n  imslp_url: https://ks15.imslp.org/files/imglnks/usimg/0/03/IMSLP576844-PMLP29989-JCdeArriag

In [16]:
obj = metadata['7383977']
imslp_id = obj['imslp'][1:]
res = requests.get(request_pdf_format % {'imslp_id': imslp_id})

In [17]:
soup = BeautifulSoup(res.text, "lxml")

In [18]:
soup.title.string

'Disclaimer - IMSLP'

In [19]:
res = requests.get(disclaimer_url_format % {'imslp_id': imslp_id})

In [20]:
soup = BeautifulSoup(res.text, "lxml")

In [24]:
res.url

'https://vmirror.imslp.org/files/imglnks/usimg/0/03/IMSLP576844-PMLP29989-JCdeArriago_Cuarteto-n3_score-part.pdf'

In [22]:
soup.title is None

True

In [27]:
pdf_url = soup.select('#sm_dl_wait')[0].get('data-id')

In [29]:
pdf_res = requests.get(pdf_url)

In [ ]:
type(pdf_res.content)